# 2. EDA

<strong>Pipeline :</strong>
1. Proteins which were missing in more than 30% samples have been removed
2. Check for the data charastricits via box plot or dot plot
3. We will decide if we should do the VSN now or not
4. We will do the QC plots for
    1. QQ plots
    2. MA plots
    3. PCA
    
<strong>Analysis TODO:</strong>
1. Analysis 1: Healthy Control Before vs. Asthma Before
2. Analysis 2: Healthy Control After vs. Asthma After

In [10]:
# Import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import yaml

# Load configuration
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Accessing config values
categorized_dir = config['datasets']['categorized_dir']
normalized_dir = config['datasets']['normalized_dir']

In [11]:
#from config we are now taking the data from catagorized and filtered data (the dir) we have specified in our yaml config file
# Construct the file paths
asthma_after = os.path.join(categorized_dir, 'df_after_asthma_filtered.csv')
asthma_before = os.path.join(categorized_dir, 'df_before_asthma_filtered.csv')
control_after = os.path.join(categorized_dir, 'df_after_control_filtered.csv')
control_before = os.path.join(categorized_dir, 'df_before_control_filtered.csv')

# Import the data
df_asthma_after = pd.read_csv(asthma_after)
df_asthma_before = pd.read_csv(asthma_before)
df_control_after = pd.read_csv(control_after)
df_control_before = pd.read_csv(control_before)

In [14]:
# for reordering the columns (proteins first) 
def reorder(df):
    last_column = df.pop('Protiens')
    df.insert(0, 'Protiens', last_column)
    return df

df_list = [
    (df_asthma_after, asthma_after), 
    (df_asthma_before, asthma_before), 
    (df_control_after, control_after), 
    (df_control_before, control_before)
]

# Reorder columns and save the dataframes back to CSV
for df, path in df_list:
    df = reorder(df)
    df.to_csv(path, index=False)

In [ ]:
#creating the plot to see the data distribution, identifing the outliers
def create_plot(df):
    # Melt the DataFrame
    df_long = pd.melt(df, id_vars='Protiens', var_name='Sample', value_name='Abundance')

    # Calculate Q1, Q2 (median), Q3 for each protein
    quartiles = df_long.groupby('Protiens')['Abundance'].quantile([0.25, 0.5, 0.75]).unstack()
    quartiles.columns = ['Q1', 'Q2', 'Q3']
    
    # Reset index to merge with original df_long
    quartiles.reset_index(inplace=True)
    df_long = pd.merge(df_long, quartiles, on='Protiens', how='left')

    # Start plotting
    plt.figure(figsize=(12, 8))
    
    # Plotting the median with error bars
    for index, row in quartiles.iterrows():
        plt.errorbar(row['Protiens'], row['Q2'], yerr=[[row['Q2']-row['Q1']], [row['Q3']-row['Q2']]], fmt='o', color='blue', capsize=5)

    # Adding a strip plot of individual points
    sns.stripplot(data=df_long, x='Protiens', y='Abundance', color='gray', alpha=0.5, jitter=True)

    plt.xticks(rotation=45)  # Rotates x-axis labels for better visibility
    plt.legend(['Median (Q2) with Q1 and Q3'])
    plt.show()

<strong> EDA is done nownow doing the EDA</strong>
1. itterate over the rows
2. take the Quatrile 1, 3 and mean of the data (across the proeien axis for the whole data set
3. Plot these 3 (Q1, mean and Q3 on a chart and also the values (for each individual protein on the y axis (this isto have a look at the data dispersion)
4. also varience of each protein needs to be checked (across the sampeles)